In [1]:
import requests
#发请求
day_sum_url="http://finance.sina.com.cn/focus/zqbjh/"
headers={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'
    }
response=requests.get(url=day_sum_url,headers=headers)

In [2]:
#将返回的结果进行储存
html_text=response.content.decode("utf-8")

In [3]:
#将文本转换成标准的html
from lxml import  etree
html=etree.HTML(html_text)

In [4]:
#解析网址
urls=html.xpath("//div/ul[@class=\"list_009\"]/li/a/@href")
#解析标题
titles=html.xpath("//div/ul[@class=\"list_009\"]/li/a/text()")
#解析发布日期
dates=html.xpath("//div/ul[@class=\"list_009\"]/li/span/text()")


In [5]:
#对网址循环发请求，返回网址内容
content=requests.get(urls[0],headers=headers).content.decode("utf-8")

In [6]:
#将内容转换成标准的html
content_html=etree.HTML(content)

In [41]:
#解析内容页的html
##解析报社列表
papers=content_html.xpath("//*[@id=\"artibody\"]/p/strong/text()")
##解析报社内容列表
news=content_html.xpath("(//*[@id=\"artibody\"]/p/text()) | (//*[@id=\"artibody\"]/p/span/a/text())")


In [42]:
#数据格式化
##删除每一个字符元素中的前后空格
def delete_space(item):
    if len(item)>2 and item.find("\u3000\u3000")==0:
        item=item.replace("\u3000\u3000","")
    return item

news=list(map(delete_space,news))


news_contents=[news_content for news_content in news 
               if (news_content.find("责任编辑")==-1) and news_content not in ['\xa0\xa0','\u3000\u3000']]

news_contents

['12月24日至25日，中央企业负责人会议在京召开。国务院国资委党委书记、主任郝鹏会中表示，明年初国企改革三年行动方案将出台实施，要以落实方案为契机推动国企改革不断实现新突破。要推进混合所有制改革实现新突破，着力引入高匹配度、高认同感、高协同性的战略投资者。',
 '银保监会网站25日消息，近日银保监会公布实施修订后的《中华人民共和国外资银行管理条例实施细则》。本次修订旨在贯彻落实银行业对外开放政策，进一步优化银行业投资和经营环境，激发外资参与',
 '中国银行',
 '业发展的活力，促进提升银行业竞争力与服务实体经济的质效。',
 '“‘稳’字当头是明年房地产市场的主基调。”接受中国证券报记者采访的专家认为，可用十个关键词观察明年楼市动向：调控政策方面是“房住不炒”“因城施策”“规范租赁市场”；行业动态方面是“房价稳中趋降”“开发投资增速回落”“土地市场冷清”“行业融资环境偏紧”；房企生存发展方面是“并购潮起”“加速转型”“掘金存量改造”。',
 '对2019年A股市场的“体感”，于散户，于机构，迥然不同。一方面，众多散户迷失在A股的反复震荡中，跑输了市场；另一方面，多数公募和私募等机构投资者，获得了超越指数的收益率。',
 '中共中央办公厅、国务院办公厅近日印发《关于促进劳动力和人才社会性流动体制机制改革的意见》。《意见》提出，全面取消城区常住人口300万以下的城市落户限制，全面放宽城区常住人口300万至500万的大城市落户条件。完善城区常住人口500万以上的超大特大城市积分落户政策，精简积分项目，确保社会保险缴纳年限和居住年限分数占主要比例。',
 '12月24日至25日，国务院国资委举行中央企业负责人会议，对2020年国企改革工作作出部署，重点工作主要包括：开展提质增效专项行动，贯彻落实国企改革三年行动方案，增强企业技术创新能力，国资调结构优布局，开展“对标一流管理提升行动”，开放合作、完善产业体系，完善以管资本为主的国有资产监管体制等八个方面。',
 '前10个月，小微企业减税1860.89亿元；民营企业合计减税10511.84亿元。今年前10个月，全国实现减税降费19688.94亿元，其中减税16473.26亿元，降低社保费3215.68亿元。',
 '昨日，人民银行、银保监会、证监会和外汇局联合发布了《关于进一步规范金融营销宣传行为的通知》。《通知》

In [ ]:
##确定报纸的顺序
real_paper= ["中国证券报","证券时报","上海证券报","证券日报"]
final_paper=[paper for paper in papers if paper in real_paper]
#新闻标题
news_titles=[paper.replace('\u3000','') for paper in papers if paper not in real_paper]


In [123]:
news=content_html.xpath("(//*[@id=\"artibody\"]/p/text()) | (//*[@id=\"artibody\"]/p/span/a/text())")
index_bug=[]
#找到有bug的新闻
for index,new in enumerate(news[:-2]):
    if (new not in ["\u3000\u3000",'\xa0']) and (news[index+1]!="\u3000\u3000"):
        index_bug.append(index)
        index_bug=list(set(index_bug))
        index_bug.append(index+1)
remember=0#动态的删除元素，需要记录
news_bug=""
for index,new in enumerate(news):
    if index in index_bug:
        news_bug+=news[index-remember]
        del news[index-remember]
        remember+=1
news.insert(min(index_bug),news_bug)

['\u3000\u3000',
 '\u3000\u3000',
 '\u3000\u300012月24日至25日，中央企业负责人会议在京召开。国务院国资委党委书记、主任郝鹏会中表示，明年初国企改革三年行动方案将出台实施，要以落实方案为契机推动国企改革不断实现新突破。要推进混合所有制改革实现新突破，着力引入高匹配度、高认同感、高协同性的战略投资者。',
 '\u3000\u3000',
 '\u3000\u3000银保监会网站25日消息，近日银保监会公布实施修订后的《中华人民共和国外资银行管理条例实施细则》。本次修订旨在贯彻落实银行业对外开放政策，进一步优化银行业投资和经营环境，激发外资参与中国银行业发展的活力，促进提升银行业竞争力与服务实体经济的质效。',
 '\u3000\u3000',
 '\u3000\u3000“‘稳’字当头是明年房地产市场的主基调。”接受中国证券报记者采访的专家认为，可用十个关键词观察明年楼市动向：调控政策方面是“房住不炒”“因城施策”“规范租赁市场”；行业动态方面是“房价稳中趋降”“开发投资增速回落”“土地市场冷清”“行业融资环境偏紧”；房企生存发展方面是“并购潮起”“加速转型”“掘金存量改造”。',
 '\u3000\u3000',
 '\u3000\u3000对2019年A股市场的“体感”，于散户，于机构，迥然不同。一方面，众多散户迷失在A股的反复震荡中，跑输了市场；另一方面，多数公募和私募等机构投资者，获得了超越指数的收益率。',
 '\u3000\u3000',
 '\u3000\u3000',
 '\u3000\u3000中共中央办公厅、国务院办公厅近日印发《关于促进劳动力和人才社会性流动体制机制改革的意见》。《意见》提出，全面取消城区常住人口300万以下的城市落户限制，全面放宽城区常住人口300万至500万的大城市落户条件。完善城区常住人口500万以上的超大特大城市积分落户政策，精简积分项目，确保社会保险缴纳年限和居住年限分数占主要比例。',
 '\u3000\u3000',
 '\u3000\u300012月24日至25日，国务院国资委举行中央企业负责人会议，对2020年国企改革工作作出部署，重点工作主要包括：开展提质增效专项行动，贯彻落实国企改革三年行动方案，增强企业技术创新能力，国资调结构优布局，开展“对标一流管理提